In [1]:
import os, shutil
import pandas as pd
import os
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import statistics
from scipy.stats.mstats import gmean
import math

In [2]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

In [3]:
import os
cwd = os.getcwd()

Cleaning up the data about the power plants 

In [4]:
gen=pd.read_csv(cwd+"//data//Generator.csv")
gen_prop=pd.read_csv(cwd+"//data//Generator_proposed.csv")
PP=pd.read_csv(cwd+"//data//PP_loc.csv")

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,9,13,14,15,17,18,34,35,36,37,38,39,40,41,42,47,48,49,50,51,52,53,62,63,67,71,72) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
Gen_tot =  pd.concat([gen, gen_prop], ignore_index=True)
Gen_tot =Gen_tot.drop_duplicates(subset=['Plant Code', 'Generator ID', ], keep='last')

In [6]:
PP.columns
PP = PP[['Plant Code', 'Plant Name', 'Latitude',
       'Longitude']]
PP_merged = pd.merge(Gen_tot, PP, on = ['Plant Name','Plant Code'])

In [7]:
PP_of_interest = 'Nuclear|Natural Gas|Coal|Onshore Wind|Solar Photovoltaic|Hydro'

Plants = PP_merged['Technology'].str.contains(PP_of_interest, case=False, na=False)
PP_merged[Plants]=PP_merged[Plants][PP_merged[Plants]['Technology'].str.contains("Other|Storage| Gasification ") == False]


In [8]:
PP_merged[Plants].columns
PP_merged= PP_merged[PP_merged['Status'].str.contains("RE|SE|SB")==False]

In [9]:
df2 = PP_merged[Plants][[ 'Plant Code', 'Generator ID', 'Plant Name', 'Latitude', 'Longitude','Technology','Nameplate Capacity (MW)', 
       'Operating Month','Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
     'Current Month', 'Current Year']]
df2=df2[df2['Technology'].notna()]
df2.columns = ['Plant Code', 'Generator ID', 'name', 'Latitude', 'Longitude','source','capacity', 
       'Operating Month','Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
     'Current Month', 'Current Year']

<ipython-input-9-cd2c6ed3aaea>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = PP_merged[Plants][[ 'Plant Code', 'Generator ID', 'Plant Name', 'Latitude', 'Longitude','Technology','Nameplate Capacity (MW)',


In [10]:
df2['Operating Year'].fillna(df2['Current Year'], inplace=True)
del df2['Current Year']
df2['Operating Month'].fillna(df2['Current Month'], inplace=True)
del df2['Current Month']

In [11]:
df2=df2.assign(coordinate= "("+df2['Latitude'].astype(str)+','+df2['Longitude'].astype(str)+")")
del df2['Latitude']
del df2['Longitude']

In [12]:
df2 = df2.replace(' ',np.nan)
df2['Planned Retirement Month'].fillna(df2['Operating Month'], inplace=True)

In [13]:
df2.loc[df2['source'].str.contains('Gas'), 'source'] = 'gas'
df2.loc[df2['source'].str.contains('Coal'), 'source'] = 'coal'
df2.loc[df2['source'].str.contains('Wind'), 'source'] = 'wind'
df2.loc[df2['source'].str.contains('Solar'), 'source'] = 'solar'
df2.loc[df2['source'].str.contains('Hydro'), 'source'] = 'hydro'
df2.loc[df2['source'].str.contains('Nuclear'), 'source'] = 'nuclear'

In [14]:
df2=df2.assign(name= df2['name'].astype(str)+'_'+df2['Generator ID'].astype(str))
del df2['Generator ID']

In [15]:
df2.name = df2.name.str.replace(' ','_')

In [16]:
def my_func(row):
    if row['source'] == 'nuclear':
        val = 60.0
    elif row['source']  == 'solar':
        val = 25.0
    elif row['source']  == 'wind':
        val = 20.0
    elif row['source']  == 'coal':
        val = 30.0        
    elif row['source']  == 'gas':
        val = 30.0          
    else:
        val = 40.0
    return val

df2['plant_life'] = df2.apply(my_func, axis=1)
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)",20.0
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)",40.0
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)",30.0
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022,10,NaN,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022,8,NaN,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023,6,NaN,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023,6,NaN,"(32.901081,-100.982695)",25.0


In [17]:
df2['Operating Year'] = df2['Operating Year'].astype(float)
df2['plant_life'] = df2['plant_life'].astype(float)
df2['Retirement'] = df2['Operating Year'] +df2['plant_life']

In [18]:
df2['Planned Retirement Year'].fillna(df2['Retirement'], inplace=True)
del df2['Retirement']
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031.0,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031.0,"(55.339722,-160.497222)",20.0
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,2003.0,"(33.458665,-87.356823)",40.0
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,1984.0,"(31.0069,-88.0103)",30.0
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,1984.0,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047.0,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022.0,8,2047.0,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048.0,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048.0,"(32.901081,-100.982695)",25.0


In [19]:
df2['Planned Retirement Year'] = df2['Planned Retirement Year'].astype(int)
plants = df2.loc[df2['Planned Retirement Year'] > 2019] 
plants['capacity'] = plants['capacity'].astype(str)
plants['capacity'] = plants['capacity'].apply(lambda x: x.replace(',', '')) 
plants['capacity'] = plants['capacity'].astype(float)

<ipython-input-19-db8401d178f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants['capacity'] = plants['capacity'].astype(str)
<ipython-input-19-db8401d178f8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants['capacity'] = plants['capacity'].apply(lambda x: x.replace(',', ''))
<ipython-input-19-db8401d178f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Create a material flow for different categories of plants 

material streams data read

In [20]:
energy_data = pd.read_csv(cwd+"//data//model_data_f.csv")
energy_data

,source,stream,material,amount (kg/MW),amount (kg/MWh),coordinate
0,gas,back_waste,aluminum,48.96,2.191780e-04,"(55.339722,-160.497222)"
1,gas,back_waste,iron,31438.00,1.407378e-01,"(55.339722,-160.497222)"
2,gas,back_waste,concrete,97749.00,4.375907e-01,"(31.0069,-88.0103)"
3,gas,front_need,iron,31438.00,1.407378e-01,"(31.0069,-88.0103)"
4,gas,front_need,concrete,97749.00,4.375907e-01,"(31.0069,-88.0103)"
...,...,...,...,...,...,...
240,nuclear,back_waste,cadmium,0.16,3.380000e-07,"(38.5857,-75.2341)"
241,nuclear,front_need,tin,1.64,3.470000e-06,"(39.807275,-104.964548)"
242,nuclear,front_need,titanium,0.01,2.110000e-08,"(39.807275,-104.964548)"
243,nuclear,front_need,zinc,2.02,4.270000e-06,"(38.2081,-104.5747)"


power plants data

In [21]:
plants

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1.0,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


# The Plan

We have to divide stations into ones that require materials for operation or not. 
Wind, solar, hydro do not require fuel constantly. They are one-shot build. Everything else requires constant refueling. 
Aggregation is annual. 

All stations build before 2019 (included) do not have any mining waste.
For those built after 2020 (included), we need to calculate mining waste. 
- How much **front_need, front_waste, back_waste** necessary?
    - Filter energy_data by stream, then group energy_data by source and aggregate over amount (kg/MW). For each plant, the front_need amount necessary is its capacity * aggregate with corresponding source.
- How much **oper_need, oper_waste** necessary?
    - Energy [MWh] = Capacity * 365 * 24 * Capacity factor [Solar = 0.25, Nuclear = 0.90, etc]. Then multiply on aggregate to amount (kg/MWh)
    - Time interval of interest in 2020- 2050. In 2020 we generates "this" much front need/ waste, oper need/waste, back need/waste.


### Generate aggregation per stream

In [22]:
streams = energy_data.loc[:, ~energy_data.columns.str.startswith('Unnamed:')].groupby(['stream', 'source'])

calculate the sum per stream (can we calculate each material amount too if needed?)

In [23]:
streams_agg = streams.sum()
streams_agg = pd.DataFrame(streams_agg.to_records())
streams_agg

,stream,source,amount (kg/MW),amount (kg/MWh)
0,back_waste,coal,2.103808e+05,1.143623
1,back_waste,gas,1.292360e+05,0.578548
2,back_waste,hydro,8.004730e+06,43.931827
3,back_waste,nuclear,2.042597e+05,0.431802
4,back_waste,solar,1.476455e+05,3.745446
5,back_waste,wind,1.746663e+04,0.398782
6,front_need,coal,2.109717e+05,1.146835
7,front_need,gas,1.293910e+05,0.579242
8,front_need,hydro,7.706588e+06,42.295555
9,front_need,nuclear,2.042597e+05,0.431802


Add amount of materials needed per MW capacity (a constant number that dependent of the source)

In [24]:
plants

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1.0,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


In [25]:
plants_capacity = plants[['name', 'source', 'capacity']]
merged = pd.merge(streams_agg, plants_capacity, on = ['source'])
merged

,stream,source,amount (kg/MW),amount (kg/MWh),name,capacity
0,back_waste,coal,210380.8467,1.143623,Whelan_Energy_Center_2,248.0
1,back_waste,coal,210380.8467,1.143623,Cross_1,590.9
2,back_waste,coal,210380.8467,1.143623,Cross_3,591.0
3,back_waste,coal,210380.8467,1.143623,Cross_4,652.0
4,back_waste,coal,210380.8467,1.143623,Seminole_(FL)_1,714.6
...,...,...,...,...,...,...
67420,oper_waste,wind,3304.0000,0.075434,"Bluestone_Wind,_LLC_BLUES",111.8
67421,oper_waste,wind,3304.0000,0.075434,Baron_Winds_II_BRNW2,113.2
67422,oper_waste,wind,3304.0000,0.075434,Number_Three_Wind_Project_65013,106.0
67423,oper_waste,wind,3304.0000,0.075434,Inertia_Wind_Project_WINER,301.0


The amount per actual capacity 

In [26]:
merged["Energy (kg/MW)"] = merged["capacity"] * merged["amount (kg/MW)"]

Amount per KWh (wrong data, may delete)

In [27]:
merged["Energy (kg/MWh)"] = merged["capacity"] * merged["amount (kg/MWh)"]

Proly, need to rename the following variables as years

In [28]:
operating_plants = plants[['name', 'source', 'capacity', 'Operating Year']]
retiring_plants = plants[['name', 'source', 'capacity', 'Planned Retirement Year']]

summarise back waste over each year for all streams

In [29]:
back = pd.merge(merged.loc[merged['stream'].isin(['back_waste'])], retiring_plants, on=['name', 'source', 'capacity']).groupby(['Planned Retirement Year', 'source']).sum()
back.drop(columns=['amount (kg/MW)', 'amount (kg/MWh)', 'capacity'], inplace=True)
back[-20:]


Energy (kg/MW)  Energy (kg/MWh)
Planned Retirement Year source                                  
2055                    solar      3.289542e+07       834.485478
2056                    gas        5.148502e+08      2304.817877
                        hydro      3.053805e+09     16759.991906
                        nuclear    2.593894e+08       548.345518
2057                    hydro      1.658580e+09      9102.674503
2058                    gas        1.422500e+08       636.807329
                        hydro      1.082240e+09      5939.582977
2059                    hydro      1.496885e+08       821.525160
2060                    hydro      1.237531e+09      6791.860416
2061                    hydro      5.211079e+08      2859.961922
2062                    hydro      2.201301e+08      1208.125236
2063                    hydro      3.562105e+08      1954.966290
2064                    hydro      6.563879e+07       360.240979
2065                    hydro      6.932096e+08      3804.496197
2066                    hydro      1.397626e+09      7670.496951
2067                    hydro      8.084778e+07       443.711450
2076                    nuclear    2.593894e+08       548.345518
2083                    nuclear    4.550906e+08       962.055133
2089                    nuclear    1.572800e+07        33.248764
2090                    nuclear    7.863998e+07       166.243818

summarise front waste over each year for all streams

In [30]:
front = pd.merge(merged.loc[merged['stream'].isin(['front_need', 'front_waste'])], operating_plants, on=['name', 'source', 'capacity']).groupby(['Operating Year', 'stream', 'source']).sum()
front.drop(columns=['amount (kg/MW)', 'amount (kg/MWh)', 'capacity'], inplace=True)
front[:4]

Energy (kg/MW)  Energy (kg/MWh)
Operating Year stream      source                                 
1901.0         front_need  hydro     1.849581e+07       101.509331
               front_waste hydro     9.330149e+06        51.206031
1904.0         front_need  hydro     1.155988e+07        63.443332
               front_waste hydro     5.831343e+06        32.003770

Start calculating the operational streams that depend on the capacity factors and capacity

In [31]:
capacity_factor = {
    "nuclear" : 0.9,
    "wind" : 0.4,
    "coal" : 0.7,
    "solar" : 0.18,
    "hydro" : 0.52,
    "gas" : 0.85,
    }

Here we calculate operational streams in KG ("Materials") based on the plant capacity and capacity factors we have mentioned above

In [32]:
streams_agg_oper = streams_agg.loc[streams_agg['stream'].isin(['oper_need', 'oper_waste'])].drop(columns=['amount (kg/MW)'])
oper_merged = pd.merge(streams_agg_oper, plants_capacity, on = ['source'])
oper_merged["Materials"] = oper_merged["capacity"] * oper_merged['amount (kg/MWh)'] * 365 * 24 * oper_merged['source'].replace(capacity_factor)
oper_merged

,stream,source,amount (kg/MWh),name,capacity,Materials
0,oper_need,coal,545.000000,Whelan_Energy_Center_2,248.0,8.288011e+08
1,oper_need,coal,545.000000,Cross_1,590.9,1.974752e+09
2,oper_need,coal,545.000000,Cross_3,591.0,1.975087e+09
3,oper_need,coal,545.000000,Cross_4,652.0,2.178945e+09
4,oper_need,coal,545.000000,Seminole_(FL)_1,714.6,2.388150e+09
...,...,...,...,...,...,...
25435,oper_waste,wind,0.075434,"Bluestone_Wind,_LLC_BLUES",111.8,2.955098e+04
25436,oper_waste,wind,0.075434,Baron_Winds_II_BRNW2,113.2,2.992102e+04
25437,oper_waste,wind,0.075434,Number_Three_Wind_Project_65013,106.0,2.801792e+04
25438,oper_waste,wind,0.075434,Inertia_Wind_Project_WINER,301.0,7.956032e+04


In [33]:
oper = pd.merge(oper_merged, operating_plants, on=['name', 'source', 'capacity'])
oper["Interest Year"] = oper.apply(lambda row: 2020 if row["Operating Year"] <= 2020.0 else row["Operating Year"], axis=1)
oper = oper.drop(columns=['Operating Year'])
oper = oper.groupby(['Interest Year', 'stream', 'source']).sum()
oper = oper.drop(columns=['capacity', 'amount (kg/MWh)'])
oper 

Materials
Interest Year stream     source               
2020.0        oper_need  coal     3.395408e+11
                         gas      5.839084e+11
                         nuclear  7.677217e+07
                         solar    1.301957e+07
                         wind     3.102208e+07
              oper_waste coal     1.837950e+12
                         gas      1.507264e+12
                         nuclear  1.208620e+10
                         solar    1.301957e+07
                         wind     3.102208e+07
2021.0        oper_need  gas      9.367592e+09
                         solar    3.684070e+06
                         wind     3.885636e+06
              oper_waste gas      2.418091e+10
                         solar    3.684070e+06
                         wind     3.885636e+06
2022.0        oper_need  gas      1.029708e+10
                         solar    6.243704e+06
                         wind     2.747183e+06
              oper_waste gas      2.658023e+10
                         solar    6.243704e+06
                         wind     2.747183e+06
2023.0        oper_need  gas      1.079399e+10
                         nuclear  1.711163e+06
                         solar    5.982411e+06
                         wind     1.033861e+06
              oper_waste gas      2.786293e+10
                         nuclear  2.693876e+08
                         solar    5.982411e+06
                         wind     1.033861e+06
2024.0        oper_need  gas      2.503085e+09
                         solar    2.930044e+06
                         wind     9.676755e+05
              oper_waste gas      6.461305e+09
                         solar    2.930044e+06
                         wind     9.676755e+05
2025.0        oper_need  gas      7.697110e+09
                         solar    4.042301e+05
                         wind     8.569254e+05
              oper_waste gas      1.986883e+10
                         solar    4.042301e+05
                         wind     8.569254e+05
2026.0        oper_need  gas      4.890757e+09
                         solar    2.895641e+05
                         wind     4.285684e+05
              oper_waste gas      1.262469e+10
                         solar    2.895641e+05
                         wind     4.285684e+05
2028.0        oper_need  gas      8.749348e+08
                         solar    1.525920e+04
              oper_waste gas      2.258501e+09
                         solar    1.525920e+04
2029.0        oper_need  nuclear  5.913805e+04
              oper_waste nuclear  9.310073e+06
2030.0        oper_need  nuclear  2.956903e+05
                         solar    6.181363e+04
              oper_waste nuclear  4.655036e+07
                         solar    6.181363e+04

## Operation materials need and waste from 2020 to 2050

In [34]:
years_of_interest = [(x, y, z) for x in range(2020, 2051) for y in ["oper_need", "oper_waste"] for z in ["coal", "gas", "nuclear", "solar", "wind"]] 
years_of_interest_dict = {}
for item in years_of_interest:
    years_of_interest_dict[item] = 0.0

oper_over_years = pd.DataFrame({"Materials over Years" : years_of_interest_dict})
oper_over_years

Materials over Years
2020 oper_need  coal                      0.0
                gas                       0.0
                nuclear                   0.0
                solar                     0.0
                wind                      0.0
...                                       ...
2050 oper_waste coal                      0.0
                gas                       0.0
                nuclear                   0.0
                solar                     0.0
                wind                      0.0

[310 rows x 1 columns]

In [35]:
plants_lifetime = plants[['name', 'source', 'capacity', 'Operating Year', 'Planned Retirement Year']]
plants_lifetime

,name,source,capacity,Operating Year,Planned Retirement Year
4,Sand_Point_WT1,wind,0.5,2011.0,2031
5,Sand_Point_WT2,wind,0.5,2011.0,2031
11,Barry_A1CT,gas,170.1,2000.0,2030
12,Barry_A1CT2,gas,170.1,2000.0,2030
13,Barry_A1ST,gas,195.2,2000.0,2030
...,...,...,...,...,...
26155,SR_Paris_SRPAR,solar,6.8,2022.0,2047
26156,SR_Jackson_II_SRJA2,solar,1.0,2022.0,2047
26157,Lumina_II_Solar_Project_LUMII,solar,357.8,2023.0,2048
26158,Lumina_Solar_Project_LUMIN,solar,357.8,2023.0,2048


In [36]:
merged_over_years = pd.merge(oper_merged, plants_lifetime, on=['name', 'source', 'capacity'])
merged_over_years.rename(columns={"Materials":"mpy", "Operating Year":"oy", "Planned Retirement Year":"pry"}, inplace=True)
merged_over_years["oy"] = merged_over_years.apply(lambda row: 2020 if row["oy"] <= 2020.0 else row["oy"], axis=1) # this is likely wrong
merged_over_years["yrs"] = merged_over_years.apply(lambda r: [year for year in range(int(r['oy']), r['pry'] + 1)], axis=1)
merged_over_years.drop(columns=['amount (kg/MWh)', 'capacity', 'oy', 'pry'], inplace=True)
merged_over_years


,stream,source,name,mpy,yrs
0,oper_need,coal,Whelan_Energy_Center_2,8.288011e+08,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
1,oper_waste,coal,Whelan_Energy_Center_2,4.486339e+09,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
2,oper_need,coal,Cross_1,1.974752e+09,"[2020, 2021, 2022, 2023, 2024, 2025]"
3,oper_waste,coal,Cross_1,1.068943e+10,"[2020, 2021, 2022, 2023, 2024, 2025]"
4,oper_need,coal,Cross_3,1.975087e+09,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
...,...,...,...,...,...
25435,oper_waste,wind,Number_Three_Wind_Project_65013,2.801792e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
25436,oper_need,wind,Inertia_Wind_Project_WINER,7.956032e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
25437,oper_waste,wind,Inertia_Wind_Project_WINER,7.956032e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
25438,oper_need,wind,Clearwater_Wind_I_CW1,9.660896e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."


In [37]:
lst = [(stream, source, mpy, yrs) for stream, source, mpy, yrs in zip(merged_over_years['stream'], merged_over_years['source'], merged_over_years['mpy'], merged_over_years['yrs'])]

# WARNING: Long runtime code below

In [38]:

for item in lst:
    stream = item[0]
    source = item[1]
    materials_per_year = item[2]
    yrs = item[3]
    for yr in yrs:
        if yr > 2050: 
            continue
        oper_over_years["Materials over Years"][yr][stream][source] += materials_per_year

In [39]:
oper_over_years

Materials over Years
2020 oper_need  coal             3.395408e+11
                gas              5.839084e+11
                nuclear          7.677217e+07
                solar            1.301957e+07
                wind             3.102208e+07
...                                       ...
2050 oper_waste coal             3.075313e+08
                gas              1.457500e+11
                nuclear          1.213973e+09
                solar            7.708670e+05
                wind             0.000000e+00

[310 rows x 1 columns]

## Calculating emissions based on mileage of delivering front_need

In [40]:
plants

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1.0,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


In [41]:
plant_coords = plants[['name', 'source', 'capacity', 'Operating Year', 'Planned Retirement Year', 'coordinate']]
plant_coords.rename(columns={"coordinate":"Plant Coordinate", "Operating Year": "oy", "Planned Retirement Year":"pry"}, inplace=True)
plant_coords["yrs"] = plant_coords.apply(lambda r: [year for year in range(int(r['oy']), r['pry'] + 1)], axis=1)
plant_coords.drop(columns=['oy', 'pry'], inplace=True)
plant_coords

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-41-ca860fcac5c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_coords["yrs"] = plant_coords.apply(lambda r: [year for year in range(int(r['oy']), r['pry'] + 1)], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,name,source,capacity,Plant Coordinate,yrs
4,Sand_Point_WT1,wind,0.5,"(55.339722,-160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
5,Sand_Point_WT2,wind,0.5,"(55.339722,-160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
11,Barry_A1CT,gas,170.1,"(31.0069,-88.0103)","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
12,Barry_A1CT2,gas,170.1,"(31.0069,-88.0103)","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
13,Barry_A1ST,gas,195.2,"(31.0069,-88.0103)","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
...,...,...,...,...,...
26155,SR_Paris_SRPAR,solar,6.8,"(36.445894,-88.327688)","[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
26156,SR_Jackson_II_SRJA2,solar,1.0,"(35.607757,-88.917046)","[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
26157,Lumina_II_Solar_Project_LUMII,solar,357.8,"(32.883528,-100.746389)","[2023, 2024, 2025, 2026, 2027, 2028, 2029, 203..."
26158,Lumina_Solar_Project_LUMIN,solar,357.8,"(32.901081,-100.982695)","[2023, 2024, 2025, 2026, 2027, 2028, 2029, 203..."


In [42]:
material_coords = energy_data
material_coords = material_coords.loc[material_coords['stream'].isin(['front_need'])]
material_coords.drop(columns=['Unnamed: 6', 'amount (kg/MWh)', 'stream'], inplace=True)
material_coords.rename(columns={"coordinate":"Material Coordinate"}, inplace=True)
material_coords

KeyError: "['Unnamed: 6'] not found in axis"

In [ ]:
coord_merged = pd.merge(plant_coords, material_coords, on=['source'])
coord_merged["Plant Coordinate"] = coord_merged["Plant Coordinate"].apply(lambda tup_s: tuple(float(x) for x in tup_s[1:-1].split(',')))
coord_merged["Material Coordinate"] = coord_merged["Material Coordinate"].apply(lambda tup_s: tuple(float(x) for x in tup_s[1:-1].split(',')))
coord_merged["Distance"] = coord_merged.apply(lambda row: geodesic(row["Plant Coordinate"], row["Material Coordinate"]).miles, axis=1)
coord_merged[:29]

,name,source,capacity,Plant Coordinate,yrs,material,amount (kg/MW),Material Coordinate,Distance
0,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",aluminum,8.026800e+03,"(36.190278, -95.2894)",3275.608462
1,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",copper,5.237760e+01,"(36.190278, -95.2894)",3275.608462
2,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",zinc,2.698240e+01,"(34.4341, -92.9049)",3456.301998
3,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",iron,4.735040e+04,"(40.45193, -121.86215)",2031.689901
4,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",concrete,2.246400e+06,"(35.462382, -93.804926)",3369.783350
5,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",copper,5.568000e+03,"(40.1719, -88.8339)",3334.873234
6,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",fiberglass,3.490800e+03,"(38.23926, -95.68978)",3160.646484
7,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",iron,5.407100e+05,"(40.570817, -122.02093)",2019.977931
8,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",paint,1.311120e+03,"(40.570817, -122.02093)",2019.977931
9,Sand_Point_WT1,wind,0.5,"(55.339722, -160.497222)","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",polyethylene,3.294000e+02,"(40.7415, -124.2103)",1928.540270


In [ ]:
coord_merged = coord_merged.drop(columns=['Plant Coordinate', 'Material Coordinate'])
coord_merged

,name,source,capacity,yrs,material,amount (kg/MW),Distance
0,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",aluminum,8.026800e+03,3275.608462
1,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",copper,5.237760e+01,3275.608462
2,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",zinc,2.698240e+01,3456.301998
3,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",iron,4.735040e+04,2031.689901
4,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",concrete,2.246400e+06,3369.783350
...,...,...,...,...,...,...,...
120004,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",germanium,4.400000e+02,2594.990604
120005,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",indium,3.800000e+02,2594.990604
120006,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",plastic,6.000000e+03,2594.990604
120007,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",lead,2.400000e+00,2594.990604


In [ ]:
def emission_factor():
    return 2

def truck_factor():
    return 1/4000

In [ ]:
coord_merged["Emission"] = emission_factor() * coord_merged["capacity"] * coord_merged["amount (kg/MW)"] * coord_merged["Distance"] * truck_factor()
# coord_merged = coord_merged.loc[coord_merged['RPY'] >= 2020]
# coord_merged["Operating Year"] = coord_merged.apply(lambda row: 2020 if row["Retirement Year"] <= 2020.0 else row["Operating Year"], axis=1)
coord_merged

,name,source,capacity,yrs,material,amount (kg/MW),Distance,Emission
0,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",aluminum,8.026800e+03,3275.608462,6.573163e+03
1,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",copper,5.237760e+01,3275.608462,4.289213e+01
2,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",zinc,2.698240e+01,3456.301998,2.331483e+01
3,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",iron,4.735040e+04,2031.689901,2.405033e+04
4,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",concrete,2.246400e+06,3369.783350,1.892470e+06
...,...,...,...,...,...,...,...,...
120004,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",germanium,4.400000e+02,2594.990604,2.169412e+03
120005,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",indium,3.800000e+02,2594.990604,1.873583e+03
120006,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",plastic,6.000000e+03,2594.990604,2.958289e+04
120007,MA-Dighton-A_MADIG,solar,3.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202...",lead,2.400000e+00,2594.990604,1.183316e+01


In [ ]:
emission_unefficient_list = [(source, yrs, emission) for source, yrs, emission in zip(coord_merged['source'], coord_merged['yrs'], coord_merged['Emission'])]

In [ ]:
emission_years_interest = [(x, y) for x in range(2020, 2051) for y in ["coal", "gas", "nuclear", "solar", "wind", "hydro"]] 
emission_years_interest_dict = {}
for item in emission_years_interest:
    emission_years_interest_dict[item] = 0.0

emission_over_years = pd.DataFrame(emission_years_interest_dict, index=[]).T
emission_over_years["Emission"] = 0.0
emission_over_years

Emission  Capacity
2020 coal          0.0       0.0
     gas           0.0       0.0
     nuclear       0.0       0.0
     solar         0.0       0.0
     wind          0.0       0.0
...                ...       ...
2050 gas           0.0       0.0
     nuclear       0.0       0.0
     solar         0.0       0.0
     wind          0.0       0.0
     hydro         0.0       0.0

[186 rows x 2 columns]

# WARNING: Long runtime code below

In [ ]:

for item in emission_unefficient_list:
    source = item[0]
    yrs = item[1]
    emission = item[2]
    for yr in yrs:
        if yr > 2050 or yr < 2020: # questionable 
            continue
        emission_over_years.loc[(yr, source)]['Emission'] += emission

In [ ]:
emission_over_years[:20]

Emission   Capacity
2020 coal     3.237844e+10   406399.6
     gas      2.370723e+10  1361442.3
     nuclear  1.475149e+10  2199126.6
     solar    5.883355e+09   516202.5
     wind     1.334702e+11  2347312.0
     hydro    6.815697e+10    44689.2
2021 coal     3.180201e+10   399159.6
     gas      2.371875e+10  1370660.7
     nuclear  1.475149e+10  2199126.6
     solar    7.838026e+09   662269.3
     wind     1.484605e+11  2640648.0
     hydro    6.372483e+10    40041.6
2022 coal     3.118589e+10   392331.2
     gas      2.385488e+10  1384882.2
     nuclear  1.475149e+10  2199126.6
     solar    1.099005e+10   909821.0
     wind     1.573955e+11  2819702.0
     hydro    6.262780e+10    39036.4
2023 coal     2.678282e+10   338503.2
     gas      2.385921e+10  1390305.9

In [ ]:
plants_years = plants[['name', 'source', 'capacity', 'Operating Year', 'Planned Retirement Year']]
plants_years.rename(columns={"Operating Year": "oy", "Planned Retirement Year":"pry"}, inplace=True)
plants_years["yrs"] = plants_years.apply(lambda r: [year for year in range(int(r['oy']), r['pry'] + 1)], axis=1)
plants_years.drop(columns=['oy', 'pry'], inplace=True)
plants_years

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_78928/1276287351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants_years.rename(columns={"Operating Year": "oy", "Planned Retirement Year":"pry"}, inplace=True)
/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_78928/1276287351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants_years["yrs"] = plants_years.apply(lambda r: [year for year in range(int(r['oy']), r['pry'] + 1)], axis=1)
/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_78928/1276287351.py:4: SettingWithCopyWa

,name,source,capacity,yrs
4,Sand_Point_WT1,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
5,Sand_Point_WT2,wind,0.5,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
11,Barry_A1CT,gas,170.1,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
12,Barry_A1CT2,gas,170.1,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
13,Barry_A1ST,gas,195.2,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
...,...,...,...,...
26155,SR_Paris_SRPAR,solar,6.8,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
26156,SR_Jackson_II_SRJA2,solar,1.0,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
26157,Lumina_II_Solar_Project_LUMII,solar,357.8,"[2023, 2024, 2025, 2026, 2027, 2028, 2029, 203..."
26158,Lumina_Solar_Project_LUMIN,solar,357.8,"[2023, 2024, 2025, 2026, 2027, 2028, 2029, 203..."


In [ ]:
cap_unefficient_list = [(source, yrs, capacity) for source, yrs, capacity in zip(plants_years['source'], plants_years['yrs'], plants_years['capacity'])]
cap_years_interest = [(x, y) for x in range(2020, 2051) for y in ["coal", "gas", "nuclear", "solar", "wind", "hydro"]] 
cap_years_interest_dict = {}
for item in cap_years_interest:
    cap_years_interest_dict[item] = 0.0

cap_over_years = pd.DataFrame(emission_years_interest_dict, index=[]).T
cap_over_years["Capacity"] = 0.0
cap_over_years


Capacity
2020 coal          0.0
     gas           0.0
     nuclear       0.0
     solar         0.0
     wind          0.0
...                ...
2050 gas           0.0
     nuclear       0.0
     solar         0.0
     wind          0.0
     hydro         0.0

[186 rows x 1 columns]

In [ ]:
for item in cap_unefficient_list:
    source = item[0]
    yrs = item[1]
    capacity = item[2]
    for yr in yrs:
        if yr > 2050 or yr < 2020:
            continue
        cap_over_years.loc[(yr, source)]['Capacity'] += capacity

In [ ]:
cap_over_years[:20]

Capacity
2020 coal     101599.9
     gas      453814.1
     nuclear   99960.3
     solar     46927.5
     wind     117365.6
     hydro     11172.3
2021 coal      99789.9
     gas      456886.9
     nuclear   99960.3
     solar     60206.3
     wind     132032.4
     hydro     10010.4
2022 coal      98082.8
     gas      461627.4
     nuclear   99960.3
     solar     82711.0
     wind     140985.1
     hydro      9759.1
2023 coal      84625.8
     gas      463435.3

## Plan

1. Calculate deltas between model and real life data, based on year and source type
2. Based on amount calculated, we need to add (build) new plants. new plant = (generic name, operation year is delta year, retirement is operation + lifetime, capacity is average of all real plants filtered on source, location is random based on source)
3. Add new recommended plants to real life plants, then run material flow on new data
4. Before running distance analysis, we need to add intermediate location, just add an extra material coordinate
5. Run distance analysis
6. Get plots for all of these


## Step 1: Calculate delta

In [ ]:
real_capacity = cap_over_years
real_cap_idx = [real_capacity.index.get_level_values(0).astype(int), real_capacity.index.get_level_values(1)]
real_capacity.set_index(real_cap_idx, inplace=True)
real_capacity.index.rename(['Interest Year', 'source'], inplace=True)
real_capacity

Capacity
Interest Year source           
2020          coal     101599.9
              gas      453814.1
              nuclear   99960.3
              solar     46927.5
              wind     117365.6
...                         ...
2050          gas       43883.1
              nuclear   10040.3
              solar      2778.5
              wind          0.0
              hydro      2601.3

[186 rows x 1 columns]

In [ ]:
model_cap_rename_dict = {'Coal' : 'coal',
                        'Gas_cc' : 'gas',
                        'Hydroelectric Power' : 'hydro',
                        'Nuclear' : 'nuclear',
                        'Solar PV' : 'solar',
                        'Wind' : 'wind'}

In [ ]:
# Read data, drop irrelevant columns, rename to be consistent with real life column names
model_capacity=pd.read_csv(cwd+"//data//capacity.csv", index_col=0).T
model_capacity.drop(columns=['Total', 'Combustion Turbine/Diesel', 'Oil and Natural Gas Steam'], inplace=True)
model_capacity.rename(columns=model_cap_rename_dict, inplace=True)

# Convert into a similar format as real life data (turn columns into multiindex)
model_capacity = model_capacity.stack().to_frame()
model_cap_dict = model_capacity[0].to_dict()
model_capacity = pd.DataFrame.from_dict({'Capacity' : model_cap_dict})

# Set the index names and types correctly
model_capacity.index.rename(['Interest Year', 'source'], inplace=True)
model_cap_idx = [model_capacity.index.get_level_values(0).astype(int), model_capacity.index.get_level_values(1)]
model_capacity.set_index(model_cap_idx, inplace=True)
model_capacity['Capacity'] = model_capacity['Capacity'].astype(float)
model_capacity['Capacity'] = model_capacity['Capacity'] * 1000

model_capacity

Capacity
Interest Year source           
2020          coal     217300.0
              gas      245400.0
              hydro     79200.0
              nuclear   97100.0
              solar     48500.0
...                         ...
2050          gas      394700.0
              hydro     79300.0
              nuclear   72300.0
              solar    351300.0
              wind     217000.0

[186 rows x 1 columns]

In [ ]:
delta = model_capacity
delta = delta.join(real_capacity, lsuffix='_model', rsuffix='_real')
delta['cap_delta'] = delta['Capacity_model'] - delta['Capacity_real']
delta.drop(columns=['Capacity_model', 'Capacity_real'], inplace=True)
delta

cap_delta
Interest Year source            
2020          coal      115700.1
              gas      -208414.1
              hydro      68027.7
              nuclear    -2860.3
              solar       1572.5
...                          ...
2050          gas       350816.9
              hydro      76698.7
              nuclear    62259.7
              solar     348521.5
              wind      217000.0

[186 rows x 1 columns]

# Step 2: Add recommended plants

In [ ]:
plants

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1.0,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


## Before we start, we need some metadata

**Plant life** is a constant lookup from dictionary.  
**Coordinate** is a random choice from all plants built on a source.   
**Capacity** is an average of itself amongst all real plants built. 


In [ ]:
plant_life_dict = {
    "wind" : 20,
    "solar" : 25,
    "nuclear" : 60,
    "hydro" : 40,
    "coal" : 30,
    "gas" : 30,
}

plants_cap_avg = plants_capacity.groupby('source').mean('capacity')

In [ ]:
# plant_coords_meta = plant_coords.groupby('source').apply(pd.DataFrame.sample, n=100).reset_index(drop=True)
# coordinate = plant_coords_meta[plant_coords_meta.source == source]['Plant Coordinate'].values
# coordinate

In [ ]:
def create_new_plant(quantity, plant_code, plant_name, source, capacity, om, oy, prm, pry, coordinate, plant_life):
    return pd.DataFrame({   'Quantity' : [quantity],
                            'Plant Code' : [plant_code],
                            'name' : [plant_name],
                            'source' : [source],
                            'capacity' : [capacity],
                            'Operating Month' : [om],
                            'Operating Year' : [oy],
                            'Planned Retirement Month' : [prm],
                            'Planned Retirement Year' : [pry],
                            'coordinate' : [coordinate],
                            'plant_life' : [plant_life]})

In [ ]:
recommended_plants = pd.DataFrame().reindex_like(plants)
recommended_plants.insert(0, "Quantity", 1)
recommended_plants = recommended_plants.iloc[0:0] # drops all data
delta_records = delta.to_records() # array of year, source and corresponding delta

plant_code = -1.0
plant_name = "Recommended Plant"
operating_month = 1
retirement_month = 12

for record in delta_records:
    operating_year = record[0]
    source = record[1]
    amount_to_cover = record[2]
    if amount_to_cover < 0:
        continue

    capacity = plants_cap_avg.loc[source][0]
    plant_life = plant_life_dict[source]
    retirement_year = operating_year + plant_life
    
    # We need to calculate number of required plants based on delta
    new_plants_ratio = amount_to_cover / capacity
    num_plants = int(math.floor(new_plants_ratio))
    k = new_plants_ratio - num_plants

    plant_coords_meta = plant_coords.groupby('source').apply(pd.DataFrame.sample, n=num_plants, replace=True).reset_index(drop=True)
    coordinate = plant_coords_meta[plant_coords_meta.source == source]['Plant Coordinate'].values
    row = create_new_plant(num_plants, plant_code, plant_name, source, capacity, operating_month, operating_year, retirement_month, retirement_year, coordinate, plant_life)
    recommended_plants = pd.concat([row, recommended_plants])

    # Resample coordinate
    plant_coords_meta = plant_coords.groupby('source').apply(pd.DataFrame.sample, n=1).reset_index(drop=True)
    coordinate = plant_coords_meta[plant_coords_meta.source == source]['Plant Coordinate'].values
    # Adding extra small plant
    extra_row = create_new_plant(1, plant_code, plant_name, source, capacity * k, operating_month, operating_year, retirement_month, retirement_year, coordinate, plant_life)
    

In [ ]:
recommended_plants

,Quantity,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
0,2158,-1.0,Recommended Plant,wind,100.516093,1.0,2050.0,12.0,2070.0,"[(41.101219,-83.644394), (36.474,-121.371), (3...",20.0
0,17918,-1.0,Recommended Plant,solar,19.449876,1.0,2050.0,12.0,2075.0,"[(39.035986,-76.054316), (41.84581,-73.619485)...",25.0
0,61,-1.0,Recommended Plant,nuclear,1016.339604,1.0,2050.0,12.0,2110.0,"[(35.211536,-120.855542), (31.2231,-85.1116), ...",60.0
0,8440,-1.0,Recommended Plant,hydro,9.086980,1.0,2050.0,12.0,2090.0,"[(43.11391,-71.894001), (44.7975,-69.8867), (4...",40.0
0,3418,-1.0,Recommended Plant,gas,102.617365,1.0,2050.0,12.0,2080.0,"[(29.716069,-95.313027), (33.5239,-101.7392), ...",30.0
...,...,...,...,...,...,...,...,...,...,...,...
0,286,-1.0,Recommended Plant,coal,395.330350,1.0,2021.0,12.0,2051.0,"[(35.1897,-81.0122), (45.3808,-93.8931), (40.4...",30.0
0,90,-1.0,Recommended Plant,wind,100.516093,1.0,2020.0,12.0,2040.0,"[(36.335277,-101.454722), (43.430639,-94.01640...",20.0
0,80,-1.0,Recommended Plant,solar,19.449876,1.0,2020.0,12.0,2045.0,"[(37.751716,-97.074497), (34.529956,-78.304811...",25.0
0,7486,-1.0,Recommended Plant,hydro,9.086980,1.0,2020.0,12.0,2060.0,"[(47.4706,-123.1153), (40.46439,-79.365703), (...",40.0


## The quantity approach works robustly

Yet, figuring out location distance might be as slow as previous approach. We will try multiprocessing.

In [ ]:
import multiprocess as mp
pool = mp.Pool(processes=mp.cpu_count())

In [ ]:
def func(args):
    plant_code, plant_name, source, capacity, operating_month, operating_year, retirement_month, retirement_year, plant_life = args
    plant_coords_meta = plant_coords.groupby('source').apply(pd.DataFrame.sample, n=1).reset_index(drop=True)
    coordinate = plant_coords_meta[plant_coords_meta.source == source]['Plant Coordinate'].values
    row = create_new_plant(1, plant_code, plant_name, source, capacity, operating_month, operating_year, retirement_month, retirement_year, coordinate, plant_life)
    return row

In [ ]:
recommended_plants_mp = pd.DataFrame().reindex_like(plants)
recommended_plants_mp.insert(0, "Quantity", 1)
recommended_plants_mp = recommended_plants_mp.iloc[0:0] # drops all data
delta_records = delta.to_records() # array of year, source and corresponding delta

plant_code = -1.0
plant_name = "Recommended Plant"
operating_month = 1
retirement_month = 12

for record in delta_records:
    operating_year = record[0]
    source = record[1]
    amount_to_cover = record[2]
    if amount_to_cover < 0:
        continue

    capacity = plants_cap_avg.loc[source][0]
    plant_life = plant_life_dict[source]
    retirement_year = operating_year + plant_life

    # We need to calculate number of required plants based on delta
    new_plants_ratio = amount_to_cover / capacity
    num_plants = int(math.floor(new_plants_ratio))
    k = new_plants_ratio - num_plants

    pool = mp.Pool(processes=mp.cpu_count())
    new_rows = pool.map(func, [(plant_code, plant_name, source, capacity, operating_month, operating_year, retirement_month, retirement_year, plant_life) 
                                for _ in range(num_plants)])
    recommended_plants_mp = pd.concat([pd.concat(new_rows), recommended_plants_mp])

    # Resample coordinate
    plant_coords_meta = plant_coords.groupby('source').apply(pd.DataFrame.sample, n=1).reset_index(drop=True)
    coordinate = plant_coords_meta[plant_coords_meta.source == source]['Plant Coordinate'].values
    # Adding extra small plant
    extra_row = create_new_plant(1, plant_code, plant_name, source, capacity * k, operating_month, operating_year, retirement_month, retirement_year, coordinate, plant_life)
    recommended_plants_mp = pd.concat([extra_row, recommended_plants_mp])


In [ ]:
recommended_plants_mp

,Quantity,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
0,1,-1.0,Recommended Plant,wind,86.270344,1.0,2050.0,12.0,2070.0,"[(18.9742,-155.6914)]",20.0
0,1,-1.0,Recommended Plant,wind,100.516093,1.0,2050.0,12.0,2070.0,"[(18.9742,-155.6914)]",20.0
0,1,-1.0,Recommended Plant,wind,100.516093,1.0,2050.0,12.0,2070.0,"[(30.772575,-102.505808)]",20.0
0,1,-1.0,Recommended Plant,wind,100.516093,1.0,2050.0,12.0,2070.0,"[(38.697606,-99.729289)]",20.0
0,1,-1.0,Recommended Plant,wind,100.516093,1.0,2050.0,12.0,2070.0,"[(42.208056,-93.599444)]",20.0
...,...,...,...,...,...,...,...,...,...,...,...
0,1,-1.0,Recommended Plant,coal,395.330350,1.0,2020.0,12.0,2050.0,"[(38.5847,-85.4117)]",30.0
0,1,-1.0,Recommended Plant,coal,395.330350,1.0,2020.0,12.0,2050.0,"[(38.205,-89.8544)]",30.0
0,1,-1.0,Recommended Plant,coal,395.330350,1.0,2020.0,12.0,2050.0,"[(41.7378,-108.7875)]",30.0
0,1,-1.0,Recommended Plant,coal,395.330350,1.0,2020.0,12.0,2050.0,"[(38.8675,-84.2289)]",30.0


### Let's save newly recommended plants into a separate csv to avoid regenerating

In [ ]:
recommended_plants_mp.to_csv('recommended_plants_mp')